In [ ]:
# ---------------
# 문제정의
# ---------------

In [1]:
# 라이브러리 가져오기

#원본데이터를 가져올때 이데이터로 무엇을할지 고민하기

In [1]:
import pandas as pd

train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [2]:
# ---------------
# EDA
# ---------------
train.shape,test.shape

((20758, 18), (13840, 17))

In [3]:
train.info()
test.info()
# 끝에 없는값이 y값이 된다.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [35]:
# 결측값 확인
train.isnull().sum()
test.isnull().sum()


Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
dtype: int64

In [36]:
# 이상치 유무 확인
train.describe()
train.describe(include='O')

ValueError: No objects to concatenate

In [6]:
# 이상치 유무 확인
test.describe()
train.describe(include='O')

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad
count,20758,20758,20758,20758,20758,20758,20758,20758,20758
unique,2,2,2,4,2,2,3,5,7
top,Female,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation,Obesity_Type_III
freq,10422,17014,18982,17529,20513,20071,15066,16687,4046


In [7]:
# train의 y값 확인
# train.info()
train["NObeyesdad"].value_counts()

NObeyesdad
Obesity_Type_III       4046
Obesity_Type_II        3248
Normal_Weight          3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level_II    2522
Overweight_Level_I     2427
Name: count, dtype: int64

In [35]:
# 분리하고 카테고리

In [ ]:
# 데이터전처리 

# 이상치, 결측치 생략

In [8]:
# id컬럼 제거
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

In [9]:
# y값분리
target = train.pop('NObeyesdad')
# dtype -> 카테고리뱐환
target = target.astype('category')
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 20758 entries, 0 to 20757
Series name: NObeyesdad
Non-Null Count  Dtype   
--------------  -----   
20758 non-null  category
dtypes: category(1)
memory usage: 20.7 KB


In [ ]:
# target/test의 object -> category로 변환

In [15]:
train.info() # 카테고리로 하나 pop헀으므로 16개로 됨
# train object에서 category로 변경
train['Gender'] = train['Gender'].astype('category')
train['family_history_with_overweight'] = train['family_history_with_overweight'].astype('category')
train['FAVC'] = train['FAVC'].astype('category')
train['CAEC'] = train['CAEC'].astype('category')
train['SMOKE'] = train['SMOKE'].astype('category')
train['SCC'] = train['SCC'].astype('category')
train['CALC'] = train['CALC'].astype('category')
train['MTRANS'] = train['MTRANS'].astype('category')
# test object에서 category로 변경
test['Gender'] = test['Gender'].astype('category')
test['family_history_with_overweight'] = test['family_history_with_overweight'].astype('category')
test['FAVC'] = test['FAVC'].astype('category')
test['CAEC'] = test['CAEC'].astype('category')
test['SMOKE'] = test['SMOKE'].astype('category')
test['SCC'] = test['SCC'].astype('category')
test['CALC'] = test['CALC'].astype('category')
test['MTRANS'] = test['MTRANS'].astype('category')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Gender                          20758 non-null  category
 1   Age                             20758 non-null  float64 
 2   Height                          20758 non-null  float64 
 3   Weight                          20758 non-null  float64 
 4   family_history_with_overweight  20758 non-null  category
 5   FAVC                            20758 non-null  category
 6   FCVC                            20758 non-null  float64 
 7   NCP                             20758 non-null  float64 
 8   CAEC                            20758 non-null  category
 9   SMOKE                           20758 non-null  category
 10  CH2O                            20758 non-null  float64 
 11  SCC                             20758 non-null  category
 12  FAF               

In [29]:
# ------------------
# 검증데이터 분할
# ------------------
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0)
X_train.shape, X_val.shape, target.shape, y_val.shape

((16606, 16), (4152, 16), (20758,), (4152,))

In [ ]:
# ------------------
# 모델 학습 및 평가
# ------------------

In [30]:
import lightgbm as lgb
# 학습
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
# 검증
pred=lgbmc.predict(X_val) # 라벨만
pred_proba=lgbmc.predict_proba(X_val) # 확률값

# vervose : 학습 과정 출력여부(-1 : 출력 안함)
# 0 : 경고 수준출력, 1: 일반정보 출력, >1 : 자세한 정보 출력

In [31]:
# 평가(roc_auc, accuracy, f1-score)
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

roc_auc_ovr = roc_auc_score(y_val, pred_proba, multi_class='ovr')
roc_auc_ovo = roc_auc_score(y_val, pred_proba, multi_class='ovo')
accuracy= accuracy_score(y_val, pred)
f1 = f1_score(y_val, pred, average='macro')

print('roc_auc_ovr : ', roc_auc_ovr)
print('roc_auc_ovo : ', roc_auc_ovo)
print('f1 : ', f1)


roc_auc_ovr :  0.9888169248146651
roc_auc_ovo :  0.9874571729788963
f1 :  0.8949765427420396


In [ ]:
#--------------------------
# 예측 파일 제출(.csv)
#--------------------------
pred_submit=lgbmc.predict(test)
pred_submit

submit_csv = pd.DataFrame({'pred' : pred_submit})
submit_csv.to_csv("result.csv",index=False)